In [1]:
import mediapipe as mp
import cv2
import os
import numpy as np
import pandas as pd

# 미디어파이프 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# 데이터 저장 리스트
pose_data = []
labels = []

# 폴더 경로 설정 (올바른 자세는 1, 틀린 자세는 0)
base_folders = {
    "right_position": 1,  # 올바른 자세
    "wrong_position": 0   # 틀린 자세
}

# 각 폴더의 데이터 전처리
for folder_name, label in base_folders.items():
    base_folder = f"C:/Users/spjni/vscode/2024-2capstone/{folder_name}"  # 각 폴더의 경로 설정
    for subfolder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, subfolder_name)
        if not os.path.isdir(folder_path):  # 폴더가 아닌 파일은 제외
            continue
        
        # 각 하위 폴더 안의 이미지 파일 처리
        for i in range(1, 33):  # 각 세트의 32 프레임
            file_name = os.path.join(folder_path, f"{subfolder_name}-{i:07d}.jpg")  # 이미지 파일 경로
            image = cv2.imread(file_name)
            if image is None:
                continue
            
            # 미디어파이프를 이용한 키포인트 추출
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                # x, y 좌표 추출
                keypoints = [landmark.x for landmark in results.pose_landmarks.landmark] + \
                            [landmark.y for landmark in results.pose_landmarks.landmark]
                pose_data.append([subfolder_name, i] + keypoints)
                labels.append(label)  # 해당 데이터의 라벨 추가

# 데이터프레임 생성 및 CSV 저장
columns = ["set_id", "frame_id"] + [f"x_{i}" for i in range(33)] + [f"y_{i}" for i in range(33)]
df = pd.DataFrame(pose_data, columns=columns)
df['label'] = labels  # 라벨 컬럼 추가
df.to_csv("overhead_press_joint_positions_with_labels.csv", index=False)


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# CSV 파일 로드 (첫 번째 코드에서 저장한 CSV 파일)
file_path = "overhead_press_joint_positions_with_labels.csv"
joint_positions = pd.read_csv(file_path)

# 시퀀스 데이터 구성 및 길이 고정
sequences = []
labels = []

for set_id in joint_positions['set_id'].unique():
    # 각 set_id에 해당하는 관절 좌표 시퀀스 추출
    sequence = joint_positions[joint_positions["set_id"] == set_id].iloc[:, 2:-1].values  # x, y 좌표만 추출
    label = joint_positions[joint_positions["set_id"] == set_id]["label"].iloc[0]  # 해당 set_id의 라벨 (0 또는 1)
    
    # 시퀀스 길이가 64보다 길 경우, 앞에서 64개 프레임만 추출
    if sequence.shape[0] > 64:
        sequence = sequence[:64]  # 64개 프레임만 사용
    elif sequence.shape[0] < 64:
        sequence = pad_sequences([sequence], maxlen=64, dtype='float32', padding='post', truncating='post')[0]

    sequences.append(sequence)
    labels.append(label)

# 시퀀스를 numpy 배열로 변환 (64 프레임, 66 좌표로 고정)
X_train = np.array(sequences)
y_train = np.array(labels)

# 데이터의 형태 확인
print("Training data shape:", X_train.shape)  # (n_samples, 64 프레임, 66 좌표)


Training data shape: (414, 64, 66)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# LSTM 모델 정의
model = Sequential([
    LSTM(64, activation='tanh', recurrent_activation='sigmoid', input_shape=(64, 66), return_sequences=False),  # 입력 크기 (64, 66)
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류 (0 또는 1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# 모델 저장 (H5 형식으로 저장)
model.save('overhead_press_model.h5')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                33536     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 35,649
Trainable params: 35,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11/11 [==============================] - 8s 43ms/step - loss: 0.6278 - accuracy: 0.9668 - val_loss: 0.4929 - val_accuracy: 1.0000
Epoch 2/10
11/11 [==============================] - 0s 13ms/step - loss: 0.2737 - accuracy: 0.9668 - val_loss: 0.0468 - val_accuracy: 1.0000
Epoch 3/10
11/11 [================

In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# CSV 파일 로드 (첫 번째 코드에서 저장한 CSV 파일)
file_path = "overhead_press_joint_positions_with_labels.csv"
joint_positions = pd.read_csv(file_path)

# 시퀀스 데이터 구성 및 길이 고정
sequences = []
labels = []

for set_id in joint_positions['set_id'].unique():
    # 각 set_id에 해당하는 관절 좌표 시퀀스 추출
    sequence = joint_positions[joint_positions["set_id"] == set_id].iloc[:, 2:-1].values  # x, y 좌표만 추출
    label = joint_positions[joint_positions["set_id"] == set_id]["label"].iloc[0]  # 해당 set_id의 라벨 (0 또는 1)
    
    # 시퀀스 길이가 64보다 길 경우, 앞에서 64개 프레임만 추출
    if sequence.shape[0] > 64:
        sequence = sequence[:64]  # 64개 프레임만 사용
    elif sequence.shape[0] < 64:
        sequence = pad_sequences([sequence], maxlen=64, dtype='float32', padding='post', truncating='post')[0]

    sequences.append(sequence)
    labels.append(label)

# 시퀀스를 numpy 배열로 변환 (64 프레임, 66 좌표로 고정)
X_train = np.array(sequences)
y_train = np.array(labels)

# 데이터의 형태 확인
print("Training data shape:", X_train.shape)  # (n_samples, 64 프레임, 66 좌표)


Training data shape: (414, 64, 66)
